In [5]:
import arcpy
import os

# === 路徑 & 參數 ===
gdb_path = r"D:\ArcGIS\Morpho_Spatial_Demography\MSD_Hawaii.gdb"
layer_t1 = os.path.join(gdb_path, "k2b_grid_4cm")
layer_t2 = os.path.join(gdb_path, "k2b_grid_4cm_2025")
key_field = "SeqID"
id_field  = "Colony_ID"
type_fields = ["up_type", "down_type", "left_type", "right_type"]

fate_field  = "Cell_fate"
ratio_field = "Cell_CPratio"   # 單一浮點數

# 只計算的代碼（排除 Self / Ncoral）
target_codes = {"br","cb","di","co","ta","ma","en","fr","fo","bu","su"}

# === 確保欄位存在且型別正確 ===
existing = {f.name: f for f in arcpy.ListFields(layer_t1)}
if fate_field not in existing:
    arcpy.AddField_management(layer_t1, fate_field, "TEXT", field_length=10)
# 若 Cell_CPratio 不是 DOUBLE，刪掉重建
if ratio_field in existing and existing[ratio_field].type.upper() != "DOUBLE":
    arcpy.DeleteField_management(layer_t1, ratio_field)
if ratio_field not in {f.name for f in arcpy.ListFields(layer_t1)}:
    arcpy.AddField_management(layer_t1, ratio_field, "DOUBLE")

# === 讀 T2 做查表 ===
t2_map = {}
with arcpy.da.SearchCursor(layer_t2, [key_field, id_field] + type_fields) as cur:
    for seq, cid, up, down, left, right in cur:
        t2_map[seq] = {"Colony_ID": cid, "up": up, "down": down, "left": left, "right": right}

# === 核心計算 ===
upd_fields = [key_field, id_field] + type_fields + [fate_field, ratio_field]
with arcpy.da.UpdateCursor(layer_t1, upd_fields) as ucur:
    for row in ucur:
        seq_id, cid_t1 = row[0], row[1]
        t1_vals = [row[2], row[3], row[4], row[5]]

        # ------- Cell_fate -------
        fate = "Neutral"
        if seq_id not in t2_map:
            fate = "Death"
        else:
            cid_t2 = t2_map[seq_id]["Colony_ID"]
            if str(cid_t1) != str(cid_t2):
                fate = "Death"
            else:
                t2_vals = [t2_map[seq_id]["up"], t2_map[seq_id]["down"],
                           t2_map[seq_id]["left"], t2_map[seq_id]["right"]]

                # Self 變化
                self_change = t2_vals.count("Self") - t1_vals.count("Self")
                # 只計入 11 個代碼的變化（排除 Self/Ncoral/空值）
                others_t1 = sum(1 for v in t1_vals if v in target_codes)
                others_t2 = sum(1 for v in t2_vals if v in target_codes)
                delta = self_change - (others_t2 - others_t1)

                if   delta > 0: fate = "Win"
                elif delta < 0: fate = "Lose"
                else:           fate = "Neutral"

        # ------- Cell_CPratio（浮點數）-------
        count_target = sum(1 for v in t1_vals if v in target_codes)
        cp_ratio = count_target / 4.0  # 分母固定 4

        # 寫回
        row[-2] = fate
        row[-1] = round(cp_ratio, 6)   # 可調整小數位
        ucur.updateRow(row)

print("✅ 已完成：Cell_fate（文字）與 Cell_CPratio（DOUBLE 浮點數）")


✅ 已完成：Cell_fate（文字）與 Cell_CPratio（DOUBLE 浮點數）


In [2]:
##加入Cell_fate_bi 與SeqID找不到對應報錯

import arcpy
import os

# === 路徑 & 參數 ===
gdb_path = r"D:\ArcGIS\Morpho_Spatial_Demography\MSD_Hawaii.gdb"
layer_t1 = os.path.join(gdb_path, "k2b_grid_4cm")
layer_t2 = os.path.join(gdb_path, "k2b_grid_4cm_2025")
key_field = "SeqID"
id_field  = "Colony_ID"
type_fields = ["up_type", "down_type", "left_type", "right_type"]

fate_field    = "Cell_fate"
ratio_field   = "Cell_CPratio"
fate_bi_field = "Cell_fate_bi"  # 新增：Colony_ID 是否相同（二元）

target_codes = {"br","cb","di","co","ta","ma","en","fr","fo","bu","su"}

# === 確保欄位存在 ===
existing = {f.name: f for f in arcpy.ListFields(layer_t1)}

# Cell_fate（文字）
if fate_field not in existing:
    arcpy.AddField_management(layer_t1, fate_field, "TEXT", field_length=10)

# Cell_CPratio（浮點）
if ratio_field in existing and existing[ratio_field].type.upper() != "DOUBLE":
    arcpy.DeleteField_management(layer_t1, ratio_field)
if ratio_field not in {f.name for f in arcpy.ListFields(layer_t1)}:
    arcpy.AddField_management(layer_t1, ratio_field, "DOUBLE")

# Cell_fate_bi（短整數）
if fate_bi_field in existing and existing[fate_bi_field].type.upper() not in ("SHORT","LONG","INTEGER"):
    arcpy.DeleteField_management(layer_t1, fate_bi_field)
if fate_bi_field not in {f.name for f in arcpy.ListFields(layer_t1)}:
    arcpy.AddField_management(layer_t1, fate_bi_field, "SHORT")

# === 讀取 T2 做查表 ===
t2_map = {}
with arcpy.da.SearchCursor(layer_t2, [key_field, id_field] + type_fields) as cur:
    for seq, cid, up, down, left, right in cur:
        t2_map[seq] = {"Colony_ID": cid, "up": up, "down": down, "left": left, "right": right}

# === 核心計算 ===
upd_fields = [key_field, id_field] + type_fields + [fate_field, ratio_field, fate_bi_field]
idx = {name: i for i, name in enumerate(upd_fields)}

with arcpy.da.UpdateCursor(layer_t1, upd_fields) as ucur:
    for row in ucur:
        seq_id, cid_t1 = row[idx[key_field]], row[idx[id_field]]
        t1_vals = [row[idx["up_type"]], row[idx["down_type"]], row[idx["left_type"]], row[idx["right_type"]]]

        # === 若 SeqID 不在 T2，直接報錯 ===
        if seq_id not in t2_map:
            raise ValueError(f"❌ 找不到 SeqID {seq_id} 在第二層資料中！")

        cid_t2 = t2_map[seq_id]["Colony_ID"]

        # --- Cell_fate_bi：只看 Colony_ID 是否改變 ---
        same_id = 1 if str(cid_t1) == str(cid_t2) else 0

        # --- 以下為原有 Cell_fate 與 Cell_CPratio ---
        fate = "Neutral"
        if same_id == 0:
            fate = "Death"
        else:
            t2_vals = [t2_map[seq_id]["up"], t2_map[seq_id]["down"],
                       t2_map[seq_id]["left"], t2_map[seq_id]["right"]]
            self_change = t2_vals.count("Self") - t1_vals.count("Self")
            others_t1 = sum(1 for v in t1_vals if v in target_codes)
            others_t2 = sum(1 for v in t2_vals if v in target_codes)
            delta = self_change - (others_t2 - others_t1)

            if   delta > 0: fate = "Win"
            elif delta < 0: fate = "Lose"
            else:           fate = "Neutral"

        count_target = sum(1 for v in t1_vals if v in target_codes)
        cp_ratio = count_target / 4.0

        # 寫回結果
        row[idx[fate_field]]     = fate
        row[idx[ratio_field]]    = round(cp_ratio, 6)
        row[idx[fate_bi_field]]  = same_id
        ucur.updateRow(row)

print("✅ 已完成：Cell_fate、Cell_CPratio、Cell_fate_bi（0/1：Colony_ID 是否一致）")


✅ 已完成：Cell_fate、Cell_CPratio、Cell_fate_bi（0/1：Colony_ID 是否一致）
